In [4]:
!pip install spacy sklearn

In [5]:
import spacy
from spacy.util import minibatch
import pandas as pd
import random

In [3]:
!git clone "https://github.com/nishithsharma9/NLUProject.git"

Cloning into 'NLUProject'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 46 (delta 5), reused 36 (delta 3), pack-reused 0
Unpacking objects: 100% (46/46), done.
Checking out files: 100% (28/28), done.


In [13]:
modelpath = "./model" #enter path where you want to save the model
datapath = "/content/NLUProject/Data/ToxicTextClassifierData"

In [7]:
nlp = spacy.blank("en")

textcat = nlp.create_pipe("textcat", config={
                                        "exclusive_classes": True,
                                        "architecture": "simple_cnn"})

In [8]:
textcat.add_label("POSITIVE")
textcat.add_label("NEGETIVE")
nlp.add_pipe(textcat, last=True)

In [9]:
dataset = pd.read_csv(datapath + "/train.csv")

In [10]:
dataset['label'] = dataset['toxic']+dataset['severe_toxic']+dataset['obscene']+dataset['threat']+dataset['insult']+dataset['identity_hate']
dataset=dataset.drop(['id','toxic', 'severe_toxic', 'obscene','threat','insult','identity_hate'], axis=1)
dataset['label']=dataset['label'].astype(bool).astype(int)

In [11]:
texts_train = dataset['comment_text'].values
labels_train = dataset['label'].values

In [12]:
# prepare the training data
labels_cats = [{'cats': {"POSITIVE": not bool(label), "NEGETIVE": bool(label)}} for label in labels_train] # 0=positive,1=negetive
data = list(zip(texts_train,labels_cats))

# train the model
optimizer = nlp.begin_training()
losses = {}
for epoch in range(5):
    random.shuffle(data)
    batches = minibatch(data,size=10)
    for batch in batches:
        texts,labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)
    


{'textcat': 12.810838584005651}


In [14]:
nlp.to_disk(modelpath)

In [15]:
## Use this saved text classifier from now
savednlp =  spacy.load(modelpath)

In [16]:
text1 = "Biological anthropology, also known as physical anthropology, is a scientific discipline concerned with the biological and behavioral aspects of human beings, their extinct hominin ancestors, and related non-human primates, particularly from an evolutionary perspective."
doc1 = savednlp(text1)
doc1.cats

{'NEGETIVE': 0.027341464534401894, 'POSITIVE': 0.9726585149765015}

In [18]:
text2 = "I hate you as you are a fucking moron"
doc2 = savednlp(text2)
doc2.cats

{'NEGETIVE': 1.0, 'POSITIVE': 4.042690071592858e-10}

In [19]:
# tweets_test = pd.read_csv(datapath + "/test.csv")

In [ ]:
# tweets_test['comment_text'].values

In [ ]:
# # evaluate the model on test data
# test_docs = list(nlp.pipe(tweets_test['comment_text'].values))
# textcat = nlp.get_pipe('textcat')
# scores, _ = textcat.predict(test_docs)
# predicted_clases = scores.argmax(axis=1)
# correct_predictions = predicted_clases==labels_test

# print("F1 score=", f1_score(predicted_clases,labels_test))
# print("ROC AUC score = ", roc_auc_score(labels_test, predicted_clases))